## Guided Project: Mobile App for Lottery Addiction

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?

- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?

- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The scenario we're following throughout this project is fictional.

### Defining Core Functions
Writing two core functions that will be used extensively during the project:

In [1]:
def factorial(n):
    product = 1
    for i in range (n, 1, -1):
        product *= i
    return product

def combinations(n, k):
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)
    return numerator / denominator

### Probability of Winning with One Ticket
For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket:

In [2]:
def one_ticket_probability(number_list):
    possible_outcomes = combinations(49, 6)
    probability = 1 / possible_outcomes
    probability_percentage = probability * 100
    message = "The probability of winning the lottery with the numbers {} is {:.7f}%. This means that you have 1 in a {:,.0f} chance of winning".format(
        number_list, probability_percentage, possible_outcomes)
    return message

Testing the function with some numbers:

In [3]:
test_1 = [1, 5, 9, 20, 40, 5]
one_ticket_probability(test_1)

'The probability of winning the lottery with the numbers [1, 5, 9, 20, 40, 5] is 0.0000072%. This means that you have 1 in a 13,983,816 chance of winning'

### Using Historical Data

For the first version of the app, however, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The data set containing historical data from 1982 to 2018 can be downloaded from [Kaggle](https://www.kaggle.com/datascienceai/lottery-dataset/data#).

In [4]:
import pandas as pd
historical = pd.read_csv('649.csv')
print(historical.shape)

(3665, 11)


In [5]:
historical.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [6]:
historical.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


### Function for Historical Data Check
- Inside the app, the user inputs six different numbers from 1 to 49.
- Under the hood, the six numbers will come as a Python list and serve as an input to our function.
- The function prints:
 - the number of times the combination selected occurred in the Canada data set;
 - the probability of winning the big prize in the next drawing with that combination.
 
We're going to begin by extracting all the winning numbers from the lottery data set. The `extract_numbers()` function will go over each row of the dataframe and extract the six winning numbers as a Python set:

In [7]:
# We extract the winning numbers from the DataFrame into sets
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

winning_numbers = historical.apply(extract_numbers, axis=1)
winning_numbers.head()

0    {3, 41, 11, 12, 43, 14}
1    {33, 36, 37, 39, 8, 41}
2     {1, 6, 39, 23, 24, 27}
3     {3, 9, 10, 43, 13, 20}
4    {34, 5, 14, 47, 21, 31}
dtype: object

Below, we write the `check_historical_occurrence()` function that takes in the user numbers and the historical numbers and prints information with respect to the number of occurrences and the probability of winning in the next drawing:

In [8]:
def check_historical_occurrence(user_numbers, winning_numbers):
    user_numbers = set(user_numbers)
    compare = (user_numbers == winning_numbers)
    total_occurrences = compare.sum()
    if total_occurrences == 0:
        print('''The combination {} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, user_numbers))
        
    else:
        print('''The number of times combination {} has occured in the past is {}.
Your chances to win the big prize in the next drawing using the combination {} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(user_numbers, total_occurrences,
                                                                            user_numbers))

We test the function:

In [9]:
check_historical_occurrence([1, 2, 3, 4, 5, 6], winning_numbers)

The combination {1, 2, 3, 4, 5, 6} has never occured.
This doesn't mean it's more likely to occur now. Your chances to win the big prize in the next drawing using the combination {1, 2, 3, 4, 5, 6} are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


### Function for Multiple Tickets Winning
Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning. We're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [32]:
def multi_ticket_probability(n_tickets):
    outcomes = combinations(49, 6)
    probability = n_tickets / outcomes
    probability_percent = probability * 100
    message = "The probability of winning the lottery with {:,} ticket(s) is {:.7f}%.".format(n_tickets, probability_percent)
    return message
    

Testing the function:

In [34]:
number_tickets = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for n in number_tickets:
    print(multi_ticket_probability(n))
    print('------------')

The probability of winning the lottery with 1 ticket(s) is 0.0000072%.
------------
The probability of winning the lottery with 10 ticket(s) is 0.0000715%.
------------
The probability of winning the lottery with 100 ticket(s) is 0.0007151%.
------------
The probability of winning the lottery with 10,000 ticket(s) is 0.0715112%.
------------
The probability of winning the lottery with 1,000,000 ticket(s) is 7.1511238%.
------------
The probability of winning the lottery with 6,991,908 ticket(s) is 50.0000000%.
------------
The probability of winning the lottery with 13,983,816 ticket(s) is 100.0000000%.
------------


### Function to Calculate the Probability of Having _n_ Winning Numbers
For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

- Inside the app, the user inputs:
 - six different numbers from 1 to 49; and
 - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

In [55]:
def probability_less_6(n_winning_numbers):
    
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(49 - n_winning_numbers,
                                           6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    n_combinations_total = combinations(49, 6)
    
    probability = successful_outcomes / n_combinations_total
    probability_percentage = probability * 100
    
    combinations_simplified = round(n_combinations_total/successful_outcomes)
    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability_percentage,
                                                               int(combinations_simplified)))

Testing the function:

In [54]:
for i in range (2, 6):
    print(probability_less_6(i))
    print('------')

Your chances of having 2 winning numbers with this ticket are 19.132653%.
In other words, you have a 1 in 5 chances to win.
None
------
Your chances of having 3 winning numbers with this ticket are 2.171081%.
In other words, you have a 1 in 46 chances to win.
None
------
Your chances of having 4 winning numbers with this ticket are 0.106194%.
In other words, you have a 1 in 942 chances to win.
None
------
Your chances of having 5 winning numbers with this ticket are 0.001888%.
In other words, you have a 1 in 52,969 chances to win.
None
------
